In [1]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
from mrcnn.model_point import log


# Directory to save logs and trained model
# MODEL_DIR = os.path.join(ROOT_DIR, "logs_point",)
MODEL_DIR = '/home/li/Mask_RCNN_point/logs_point_test/res50_FF_hour256_8_0.0005_b2_k9/guidewire20210512T0016'
sys.path.append(MODEL_DIR)
import guidewire_config as guidewire
import model as modellib
config = guidewire.GuidewireConfig()
guidewire_DIR = '/home/li/GuideWire/guidewire_2020'
class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()

/home/li/anaconda3/envs/keras/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/li/anaconda3/envs/keras/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/li/anaconda3/envs/keras/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/li/anaconda3/envs/keras/lib/python3.6/site-packages/ten


Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
BRANCH                         hour
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        5
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
DOUBLE                         False
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  512
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              none
IMAGE_SHAPE                    [512 512   3]
INFERENCE_NUM                  200
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.0005
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'class_loss': 1.0, 'bbox_l

Using TensorFlow backend.


In [2]:
# Load validation dataset
dataset = guidewire.GuidewireDataset()
dataset.load_guidewire(guidewire_DIR, "val")

# Must call before using the dataset
dataset.prepare()
print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

Images: 1041
Classes: ['BG', 'guidewire']


In [3]:
DEVICE = "/gpu:0"
TEST_MODE = "inference"
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode=TEST_MODE, model_dir=MODEL_DIR,
                              config=config)
# Set path to weights file
def find_last1(dir_name):
        """Finds the last checkpoint file of the last trained model in the
        model directory.
        """
        # Find the last checkpoint
        checkpoints = next(os.walk(dir_name))[2]
        checkpoints = filter(lambda f: f.startswith("mask_rcnn"), checkpoints)
        checkpoints = sorted(checkpoints)
        if not checkpoints:
            import errno
            raise FileNotFoundError(
                errno.ENOENT, "Could not find weight files in {}".format(dir_name))
        checkpoint = os.path.join(dir_name, checkpoints[-1])
        return checkpoint    

# Or, load the last model you trained
weights_path = find_last1(MODEL_DIR)
# weights_path = "/home/li/Mask_RCNN_point/logs_point1/resnet_aug_False_oldall_p2_slim/guidewire20201015T1528/mask_rcnn_guidewire_0001.h5"
# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

resnet50


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.
Loading weights  /home/li/Mask_RCNN_point/logs_point_test/res50_FF_hour256_8_0.0005_b2_k9/guidewire20210512T0016/mask_rcnn_guidewire_0080.h5
Re-starting from epoch 80


In [8]:
# AP
import compute_ap
MAP = compute_ap.MAPCallback(model, dataset, ['tip','back'], threshold=7, inference_num=1041)
# MAP = compute_ap.MAPCallbackSame(model, dataset, ['tip'], threshold=7, inference_num =1041)
MAP.on_epoch_end()

avg_infer_time:0.13483106391459676
tip ap:  0.5396419960316137
back ap:  0.5740403738240891
mAP:  0.5568411849278514
tip ap:  0.7852213758768757
back ap:  0.8793382304044728
mAP:  0.8322798031406742
tip ap:  0.8468529623494809
back ap:  0.9132211664550691
mAP:  0.880037064402275
tip ap:  0.8731502521325687
back ap:  0.9228108011129961
mAP:  0.8979805266227824


In [4]:
import compute_ap
import imp
imp.reload(compute_ap)
# TXT_DIR = '/home/li/GuideWire/guidewire_2020/txt'
pic_filename = '/home/li/GuideWire/guidewire_2020/val'
output_dir = '/home/li/Mask_RCNN_point/result_1'
compute_ap.make_output(pic_filename, model, output_dir)

In [ ]:
# detection
import compute_ap
import imp
imp.reload(compute_ap)
MAP = compute_ap.MAPCallbackBox(model, dataset, ['guidewire'], inference_num=1041)
MAP.on_epoch_end()

In [ ]:
# AP_PCK
import compute_ap
DEVICE = "/gpu:0"
TEST_MODE = "point_inference"
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode=TEST_MODE, model_dir=MODEL_DIR,
                              config=config)
# Set path to weights file
def find_last1(dir_name):
        """Finds the last checkpoint file of the last trained model in the
        model directory.
        """
        # Find the last checkpoint
        checkpoints = next(os.walk(dir_name))[2]
        checkpoints = filter(lambda f: f.startswith("mask_rcnn"), checkpoints)
        checkpoints = sorted(checkpoints)
        if not checkpoints:
            import errno
            raise FileNotFoundError(
                errno.ENOENT, "Could not find weight files in {}".format(dir_name))
        checkpoint = os.path.join(dir_name, checkpoints[-1])
        return checkpoint    

# Or, load the last model you trained
weights_path = find_last1(MODEL_DIR)
# weights_path = "/home/li/Mask_RCNN_point/logs_point1/resnet_aug_False_oldall_p2_slim/guidewire20201015T1528/mask_rcnn_guidewire_0001.h5"
# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)
MAP = compute_ap.MAPCallbackPCK(model, dataset, ['tip','back'], inference_num=1041)
MAP.on_epoch_end()

In [6]:
import compute_ap
MAP = compute_ap.MAPCallbackMask(model, dataset, ['tip'], threshold=0.05, inference_num =1041)
MAP.on_epoch_end()

avg_infer_time:0.13407498271733265
tip ap:  0.3981903929880875
mAP:  0.3981903929880875
tip ap:  0.6881625300022318
mAP:  0.6881625300022318
tip ap:  0.752650798917317
mAP:  0.752650798917317
tip ap:  0.7762002126063311
mAP:  0.7762002126063311
